# **DATOS DE DETECCIÓN DE MASCARILLA**





In [1]:
%tensorflow_version 1.x
import tensorflow as tf 
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


## **1. Configuración de rutas**

Se configuran las rutas necesarias para faciltar su cambio en todo el código de ser necesario. 

In [2]:
#CUSTOM_MODEL_NAME = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19'
CUSTOM_MODEL_NAME = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18'

APIMODEL_PATH = 'models'
WORKSPACE_PATH = 'workspace/MASK'
MODEL_PATH = WORKSPACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
CHECKPOINT_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME

IMAGE_PATH = '/content/carga/MyDrive/workspace/images-todo'
ANNOTATION_PATH = '/content/carga/MyDrive/workspace/annotations' 
OUTPUT_PATH = '/content/carga/MyDrive/workspace/tf1/'+ CUSTOM_MODEL_NAME + '/output'
EXPORT_PATH = '/content/carga/MyDrive/workspace/tf1/'+ CUSTOM_MODEL_NAME + '/export'

## **2. Carga y separación de datos**

Cargamos las imágenes y los XML generados con LabelImg, en esta ocación las cargamos de nuestro directorio en Drive ya que es mucho mas fácil y rápido de importar.

In [3]:
from google.colab import drive
import os
drive.mount('carga')

Mounted at carga


Ahora separaremos los datos que usaremos para el entrenamiento y los que se usarán para las validaciones. (para cambiar la relación con la se separan es necesario cambiar la variable **ratio**)

In [4]:
import os
import re
from shutil import copyfile
import math
import random

source = IMAGE_PATH
dest = source
ratio = 0.2
copy_xml = True

source = source.replace('\\', '/')
dest = dest.replace('\\', '/')
train_dir = os.path.join(dest, 'train')
test_dir = os.path.join(dest, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

images = [f for f in os.listdir(source) if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

num_images = len(images)
num_test_images = math.ceil(ratio*num_images)

for i in range(num_test_images):
    idx = random.randint(0, len(images)-1)
    filename = images[idx]
    copyfile(os.path.join(source, filename), os.path.join(test_dir, filename))
    if copy_xml == True:
        xml_filename = os.path.splitext(filename)[0]+'.xml'
        copyfile(os.path.join(source, xml_filename), os.path.join(test_dir,xml_filename))
        images.remove(images[idx])

for filename in images:
    copyfile(os.path.join(source, filename), os.path.join(train_dir, filename))
    if copy_xml == True:  
        xml_filename = os.path.splitext(filename)[0]+'.xml'
        copyfile(os.path.join(source, xml_filename), os.path.join(train_dir, xml_filename))

## **5. Crear mapas de etiquetas (Label_Map)**

In [5]:
import os

labels = [{'name':'Mask', 'id':1}, {'name':'No_Mask', 'id':2}]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
  for label in labels:
    f.write('item { \n')
    f.write('\tname:\'{}\'\n'.format(label['name']))
    f.write('\tid:{}\n'.format(label['id']))
    f.write('}\n')

print('OK')

OK


## **3. Instalar la API de detección de objetos de TensorFlow**

In [6]:
import os
import pathlib

# Clonar el repositorio 
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone https://github.com/tensorflow/models.git
  

Cloning into 'models'...
remote: Enumerating objects: 494, done.
remote: Counting objects: 100% (494/494), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 55242 (delta 268), reused 414 (delta 219), pack-reused 54748
Receiving objects: 100% (55242/55242), 571.89 MiB | 38.26 MiB/s, done.
Resolving deltas: 100% (37924/37924), done.


In [7]:
%%bash
cd models/research
pip install --upgrade pip
# Compilar protos
protoc object_detection/protos/*.proto --python_out=.
# Instalar la API TensorFlow Object Detection
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1643258 sha256=faf23efdea98dc0e4aba97c79f5ca0161b71a0d7fb4c2d12df3233ebfd154ef2
  Stored in directory: /tmp/pip-ephem-wheel-cache-1qo21ooo/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection


In [8]:
!python models/research/object_detection/builders/model_builder_tf1_test.py

Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_w

In [ ]:
!ls

carga  models  sample_data


## **6. Crear archivos TF (TFRecords)**

In [9]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET


#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple


labels_path = ANNOTATION_PATH + '/label_map.pbtxt'

label_map = label_map_util.load_labelmap(labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
          
                     int(root.find('size').find('width').text),
                     int(root.find('size').find('height').text),
                     member[0].text,
                     int(member.find("bndbox").find('xmin').text),
                     int(member.find("bndbox").find('ymin').text),
                     int(member.find("bndbox").find('xmax').text),
                     int(member.find("bndbox").find('ymax').text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'png'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main(_):
    lib = ['train', 'test']
    
    for n in lib:

        xml_dir = IMAGE_PATH + '/' + n
        output_path = ANNOTATION_PATH + '/' + n + '.record'
        
        image_dir = xml_dir
        csv_path = ANNOTATION_PATH + '/' + n + '.cvs'

        writer = tf.python_io.TFRecordWriter(output_path)
        path = os.path.join(image_dir)
        examples = xml_to_csv(xml_dir)
        grouped = split(examples, 'filename')
        for group in grouped:
            tf_example = create_tf_example(group, path)
            writer.write(tf_example.SerializeToString())
        writer.close()
        print('Successfully created the TFRecord file: {}'.format(output_path))
        if csv_path is not None:
            examples.to_csv(csv_path, index=None)
            print('Successfully created the CSV file: {}'.format(csv_path))

if __name__ == '__main__':
    tf.app.run()

Successfully created the TFRecord file: /content/carga/MyDrive/workspace/annotations/train.record
Successfully created the CSV file: /content/carga/MyDrive/workspace/annotations/train.cvs
Successfully created the TFRecord file: /content/carga/MyDrive/workspace/annotations/test.record
Successfully created the CSV file: /content/carga/MyDrive/workspace/annotations/test.cvs


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
